This example shows how to collect paginated news headlines using AgentQL with the [Google Colaboratory](https://colab.research.google.com/) environment.

Install [AgentQL](https://pypi.org/project/agentql/) library

In [ ]:
%pip install agentql

Install the Playwright dependency required by AgentQL.

In [ ]:
!playwright install chromium

You can [store](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) keys in Google Colab's secrets.

In [ ]:
import os

from google.colab import userdata

os.environ["AGENTQL_API_KEY"]=userdata.get('AGENTQL_API_KEY')

Run AgentQL script. Unfortunately, you can't see the browser window directly in Google Colab like you would on your local machine. However, let's still interact with it and take a screen recording of the browser session to see what’s happening. 

Please note that an online environment like Google Colab supports **asynchronous version** of AgentQL.

In [ ]:
import json
import logging

import agentql
from playwright.async_api import async_playwright
from IPython.display import HTML
from base64 import b64encode

# import paginate tool from agentql tools
from agentql.tools.async_api import paginate

logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)


async with async_playwright() as playwright, await playwright.chromium.launch() as browser:
      # Set up the video recording
      video_dir = os.path.abspath("videos")
      context = await browser.new_context(
        record_video_dir="videos/",
        record_video_size={"width": 1280, "height": 720}
        )
      page = await agentql.wrap_async(await context.new_page())
      await page.goto("https://news.ycombinator.com/")

      # Define the query to extract post titles
      QUERY = """
      {
          posts[] {
              title
          }
      }
      """
      # Collect all data over the next 3 pages with the query defined above
      paginated_data = await paginate(page, QUERY, 3)

      # Save the aggregated data to a json file
      with open("./hackernews_paginated_data.json", "w") as f:
          json.dump(paginated_data, f, indent=4)

      log.debug("Paginated data has been saved to hackernews_paginated_data.json")

      # Display the video
      video_files = [f for f in os.listdir(video_dir) if f.endswith('.webm')]
      if video_files:
          video_path = os.path.join(video_dir, video_files[0])
          with open(video_path, 'rb') as f:
            video_bytes = f.read()

          video_b64 = b64encode(video_bytes).decode('utf-8')
          video_html = f"""
          <video width="800" controls>
            <source src="data:video/webm;base64,{video_b64}" type="video/webm">
            Your browser does not support the video tag.
          </video>
          """
          display(HTML(video_html))
      else:
          print("No video file was created")